In [1]:
from astropy.table import Table, join
import matplotlib.pyplot as plt
import numpy as np

In [2]:
b3 = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_b3_catalog_bgfit_may21.fits') #b3 huge catalog new
b6 = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_b6_catalog_bgfit_may21_refit.fits')
b7 = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_b7_catalog_bgfit_may21_refit.fits')
old_catalog = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim_mask.fits')

TO DO:
    -remove low S/N B3 sources
    -remove B6 non detections
    -add ID column to b3 table - reorder so it's in front
    -add D_ID column for backwards compatibility 
    -match tables
    -take care of masked values so they are all nan rather than some weird number  
    -add flux upper limits, and size upper limits   
        -create 'mask' version with the deconvolved fits of 113,114,124,125 masked

In [4]:
##first remove nondetections from b6 table
remove_b6 = np.array([3,14,17,42,61])
for seq in remove_b6:
    ind = np.where(b6['B3_Seq'] == seq)
    b6.remove_row(ind[0][0])

In [10]:
low_sn_ind = np.where(b3['SNR_B3'] < 5)
b3.remove_rows(low_sn_ind)

In [11]:
print(b6[np.where(b6['SNR_B6'] < 3)])
print(b7[np.where(b7['SNR_B7'] < 3)])
print(b3[np.where(b3['SNR_B3'] < 5)])

B3_Seq fwhm_maj_B6 fwhm_maj_err_B6 fwhm_min_B6 ... SNR_B6 RMS_B6 ar_deconv_B6
------ ----------- --------------- ----------- ... ------ ------ ------------
B3_Seq fwhm_maj_B7 fwhm_maj_err_B7 fwhm_min_B7 ... SNR_B7 RMS_B7 ar_deconv_B7
------ ----------- --------------- ----------- ... ------ ------ ------------
B3_Seq fwhm_maj_B3 fwhm_maj_err_B3 fwhm_min_B3 ... SNR_B3 RMS_B3 ar_deconv_B3
------ ----------- --------------- ----------- ... ------ ------ ------------


In [18]:
#add ID column:
b3['ID'] = np.arange(len(b3))
col_order = b3.colnames
col_order_new = np.concatenate((['ID'], col_order[:-1]))
print(col_order_new)
b3 = b3[list(col_order_new)]

['ID' 'B3_Seq' 'fwhm_maj_B3' 'fwhm_maj_err_B3' 'fwhm_min_B3'
 'fwhm_min_err_B3' 'pa_B3' 'pa_err_B3' 'gauss_amp_B3' 'gauss_amp_err_B3'
 'RA_B3' 'RA_err_B3' 'DEC_B3' 'DEC_err_B3' 'ap_flux_B3' 'ap_flux_err_B3'
 'fwhm_maj_deconv_B3' 'fwhm_maj_deconv_err_B3' 'fwhm_min_deconv_B3'
 'fwhm_min_deconv_err_B3' 'pa_deconv_B3' 'pa_deconv_err_B3' 'SNR_B3'
 'RMS_B3' 'ar_deconv_B3']


In [87]:
#adding D_ID column:
ref = Table.read('/home/jotter/nrao/summer_research_2018/tables/ref_catalog_may21.fits')
did_tab = Table((ref['B3_Seq'], ref['D_ID']))
b3join = join(b3, did_tab, keys='B3_Seq', join_type='left')
bad_did = np.where(b3join['D_ID'].data > 1e5)[0]
b3join['D_ID'][bad_did] = -1
bad_did2 = np.where(b3join['D_ID'].data < -1e5)[0]
b3join['D_ID'][bad_did2] = -1

In [88]:
#matching tables
b3b6 = join(b3join, b6, keys='B3_Seq', join_type='left')
b3b6b7 = join(b3b6, b7, keys='B3_Seq', join_type='outer')

In [89]:
for col in b3b6b7.colnames:
    b3b6b7[col].fill_value = np.nan

In [90]:
b3b6b7['fwhm_maj_B7'].fill_value

nan

In [92]:
b3b6b7.write('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21.fits', overwrite=True)

In [93]:
##added size and flux upper limits in other scripts, now load in table
tab = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim.fits')

In [95]:
badfit_ind = np.array([113,114,124,125])
for col in tab.colnames: 
    if col[-9:] == 'deconv_B3' or col[-13:] == 'deconv_err_B3':
        print(col)
        tab[col][badfit_ind] = np.nan

fwhm_maj_deconv_B3
fwhm_maj_deconv_err_B3
fwhm_min_deconv_B3
fwhm_min_deconv_err_B3
pa_deconv_B3
pa_deconv_err_B3
ar_deconv_B3


In [96]:
tab.write('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim_mask.fits', overwrite=True)

In [99]:
tab

ID,B3_Seq,fwhm_maj_B3,fwhm_maj_err_B3,fwhm_min_B3,fwhm_min_err_B3,pa_B3,pa_err_B3,gauss_amp_B3,gauss_amp_err_B3,RA_B3,RA_err_B3,DEC_B3,DEC_err_B3,ap_flux_B3,ap_flux_err_B3,fwhm_maj_deconv_B3,fwhm_maj_deconv_err_B3,fwhm_min_deconv_B3,fwhm_min_deconv_err_B3,pa_deconv_B3,pa_deconv_err_B3,SNR_B3,RMS_B3,ar_deconv_B3,D_ID,fwhm_maj_B6,fwhm_maj_err_B6,fwhm_min_B6,fwhm_min_err_B6,pa_B6,pa_err_B6,gauss_amp_B6,gauss_amp_err_B6,RA_B6,RA_err_B6,DEC_B6,DEC_err_B6,ap_flux_B6,ap_flux_err_B6,fwhm_maj_deconv_B6,fwhm_maj_deconv_err_B6,fwhm_min_deconv_B6,fwhm_min_deconv_err_B6,pa_deconv_B6,pa_deconv_err_B6,SNR_B6,RMS_B6,ar_deconv_B6,fwhm_maj_B7,fwhm_maj_err_B7,fwhm_min_B7,fwhm_min_err_B7,pa_B7,pa_err_B7,gauss_amp_B7,gauss_amp_err_B7,RA_B7,RA_err_B7,DEC_B7,DEC_err_B7,ap_flux_B7,ap_flux_err_B7,fwhm_maj_deconv_B7,fwhm_maj_deconv_err_B7,fwhm_min_deconv_B7,fwhm_min_deconv_err_B7,pa_deconv_B7,pa_deconv_err_B7,SNR_B7,RMS_B7,ar_deconv_B7,upper_lim_B3,upper_lim_B6,upper_lim_B7,B6_flux_ulim,B7_flux_ulim
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,AU,AU,AU,,
int64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0,0,0.10305585997928254,0.005978321021454948,0.08478250887162492,0.004918503770637874,56.95452545532544,0.20888564676048593,0.0002547371545014736,1.4777247800786749e-05,83.81515090326748,6.704696865522927e-07,-5.3823407933457466,6.200124333207998e-07,0.00030326626340923296,7.005527787363965e-06,0.05180421201112408,0.005978321021454948,0.026503408218873602,0.004918503770637874,-1.2702585309348218,0.20888564676048593,19.399299361304976,1.4112799362919759e-05,1.9546245367127264,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1,0.11584944756560857,0.014047580955655387,0.06481658989387087,0.007859839676357889,58.66559214544469,0.13968770465164923,0.0001731617899034192,2.0999984799853715e-05,83.80704254391247,1.4952667586795851e-06,-5.382287866738019,1.170309120265059e-06,0.00016871907851167075,1.4191411921993958e-05,nan,nan,nan,nan,nan,nan,7.024454045060415,1.4553046639775857e-05,nan,3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,18.110688028549117,nan,nan,nan,nan
2,2,0.07598532943911104,0.008706835425854306,0.0637818541430692,0.00730782438216444,241.73067686299476,0.46020146578238064,0.00022885897305836324,2.62237110154526e-05,83.8169899462259,9.924663864372184e-07,-5.381757079653479,9.016615753801123e-07,0.0001547257992428559,1.3263358536929322e-05,nan,nan,nan,nan,nan,nan,12.371471226668866,1.596290894667618e-05,nan,4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,13.646791700015182,nan,nan,nan,nan
3,3,0.125214056844178,0.0014964199953616476,0.08549277663331106,0.0010217160868824608,35.61556476235245,0.021617061991398567,0.0011603951895139634,1.3867723890371504e-05,83.80986005156173,1.4200913628479887e-07,-5.381681607707714,1.597448655507702e-07,0.0015933513090232746,5.678990436011385e-06,0.08054361738385311,0.0014964199953616476,0.04558314961964404,0.0010217160868824608,0.4908353078371169,0.021617061991398567,98.41361084466187,1.1791003089456353e-05,1.

In [15]:
#adding fit for source 41, which I somehow left out before
b6_tab = Table.read('/home/jotter/nrao/tables/src41_b6fit.fits')
#tab = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21.fits')
#tab = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim.fits')
tab = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim_mask.fits')

for col in b6_tab.colnames: #these are nondetections that I accidentally didnt get rid of before
    if col[-2:] == 'B6':
        tab[col][41] = b6_tab[col]

In [16]:
tab[41]

ID,B3_Seq,fwhm_maj_B3,fwhm_maj_err_B3,fwhm_min_B3,fwhm_min_err_B3,pa_B3,pa_err_B3,gauss_amp_B3,gauss_amp_err_B3,RA_B3,RA_err_B3,DEC_B3,DEC_err_B3,ap_flux_B3,ap_flux_err_B3,fwhm_maj_deconv_B3,fwhm_maj_deconv_err_B3,fwhm_min_deconv_B3,fwhm_min_deconv_err_B3,pa_deconv_B3,pa_deconv_err_B3,SNR_B3,RMS_B3,ar_deconv_B3,D_ID,fwhm_maj_B6,fwhm_maj_err_B6,fwhm_min_B6,fwhm_min_err_B6,pa_B6,pa_err_B6,gauss_amp_B6,gauss_amp_err_B6,RA_B6,RA_err_B6,DEC_B6,DEC_err_B6,ap_flux_B6,ap_flux_err_B6,fwhm_maj_deconv_B6,fwhm_maj_deconv_err_B6,fwhm_min_deconv_B6,fwhm_min_deconv_err_B6,pa_deconv_B6,pa_deconv_err_B6,SNR_B6,RMS_B6,ar_deconv_B6,fwhm_maj_B7,fwhm_maj_err_B7,fwhm_min_B7,fwhm_min_err_B7,pa_B7,pa_err_B7,gauss_amp_B7,gauss_amp_err_B7,RA_B7,RA_err_B7,DEC_B7,DEC_err_B7,ap_flux_B7,ap_flux_err_B7,fwhm_maj_deconv_B7,fwhm_maj_deconv_err_B7,fwhm_min_deconv_B7,fwhm_min_deconv_err_B7,pa_deconv_B7,pa_deconv_err_B7,SNR_B7,RMS_B7,ar_deconv_B7,upper_lim_B3,upper_lim_B6,upper_lim_B7,B6_flux_ulim,B7_flux_ulim
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,AU,AU,AU,,
int64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
41,41,0.10506711313214062,0.001655787002007198,0.08544549420012472,0.0013465595157379307,40.748531304022265,0.05352349029527983,0.0007797641136222767,1.2288611564901236e-05,83.80726447594998,1.7531284550416592e-07,-5.3727749436508505,1.806810816594237e-07,0.0009069323141098084,5.204563707446018e-06,0.04774242527039455,0.001655787002007198,0.04047486272984116,0.0013465595157379307,-0.5331879578416203,0.05352349029527983,78.90163739636097,9.775051694305148e-06,1.1795574351681541,46,0.14154948323279043,0.006418096297410904,0.10517067376895163,0.0047684924561777866,2.592761622605398,0.10634766451734487,0.0014415627455892275,6.536473647212353e-05,83.80726259554115,7.567385394643028e-07,-5.372776018232102,5.629657540548359e-07,0.027470672550397614,8.1110983891764e-06,0.11653920733450066,0.006418096297410904,0.05561710996631059,0.0047684924561777866,-1.3185510027272271,0.10634766451734487,23.512672437593064,nan,2.095384089628046,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [17]:
#tab.write('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim_mask.fits', overwrite=True)

In [20]:
len(np.where(np.isnan(tab['ap_flux_B6'])==False)[0]), len(np.where(np.isnan(tab['B6_flux_ulim'])==False)[0])

(50, 9)

In [30]:
tab = Table.read('/home/jotter/nrao/summer_research_2018/tables/r0.5_catalog_bgfit_may21_ulim.fits')

In [31]:
tab['fwhm_maj_deconv_B6'][41]

0.11653920733450066